In [6]:
import numpy as np ; import pandas as pd ; import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
df_l = pd.read_csv("./data/lyrics.csv")
df_g = pd.read_csv("./data/songsbyGenrePages.csv")

In [8]:
def get_ID(song_URL):
    x = re.search(r"(\/lyric\/)(\d+)\/.*$", song_URL)
    if x :
      return(x.group(2))

In [9]:
def data_lyrics(df_lyrics, df_genres, output, tfid = False, ngram = 1):
    if output not in [1, 2, 3]:
        print("Wrong output type")
        exit
    else :
        pass
    df_copy = df_lyrics.copy()
    df_copy.columns = ['artist_name', 'artist_url', 'song_title', 'song_url', 'song_lyrics','song_runtime', 'song_year']
    df_genres.columns = ['song_title', 'song_url', 'song_genre']

    # Remove songs with no lyrics
    df_copy['song_lyrics'].drop(df_copy[df_copy['song_lyrics'].str.contains("Not found")==True].index, inplace = True)
    
    # Keep Only letters and space 
    # df_copy["song_lyrics"].str.encode('utf-8').str.decode('ascii', 'ignore')
    df_copy['song_lyrics'] = df_copy['song_lyrics'].astype(str)
    df_copy["song_title"] =  [re.sub(r'[^A-Za-z0-9 ]','', str(x)) for x in df_copy["song_title"] ]
    df_copy["song_lyrics"] =  [re.sub(r'[^A-Za-z0-9 ]','', str(x)) for x in df_copy["song_lyrics"] ]
    df_copy["song_lyrics"] = df_copy["song_lyrics"].str.lower()
    df_copy["song_title"] = df_copy["song_title"].str.lower()
    df_copy["artist_name"] = df_copy["artist_name"].str.lower()
    
    # Check if there are some NaN values left and remove them
    # df_copy.dropna(inplace = True)

    # Make song's runtime time type
    # df_copy.drop(df_copy[df_copy['song_runtime'] == '0'].index, inplace = True)
    df_copy.loc[df_copy['song_runtime'] == '0', 'song_runtime'] = '00:00:00'
    df_copy.loc[df_copy['song_runtime'] == '-1', 'song_runtime'] = '00:30:00'
    df_copy['song_runtime'] = pd.to_datetime(df_copy['song_runtime']).dt.time
   
    # Make song's year year type
    df_copy.loc[df_copy['song_year'].isnull() == True, 'song_year'] = 0
    df_copy['song_year'] = df_copy['song_year'].astype(int)

    # Create a word count column 
    df_copy['song_word_count'] = df_copy['song_lyrics'].str.split().str.len()
    df_copy['song_word_count'].astype(int)

    # Remove duplicates
    df_copy.drop_duplicates(['song_lyrics'], inplace = True)
    # Merge genres and lyrics
    # attach IDs to dataframes
    df_genres['song_ID'] = df_genres['song_url'].apply(get_ID)
    df_copy['song_ID'] = df_copy['song_url'].apply(get_ID)
    df_copy = df_copy.merge(df_genres, on="song_ID", how = "left")
    df_copy.rename({'song_title_x' : 'song_title'}, axis = 1, inplace = True)

    df_copy.drop(['song_url_x','song_url_y', 'artist_url', 'song_ID', 'song_title_y'], axis=1, inplace=True)
    # Create output
    if output == 1:
        df_copy.drop(['song_lyrics'], axis=1, inplace=True)
        df_copy.to_csv('model_input_covariates.csv', index = False)
    elif output == 2 and tfid == False:
        vectorizer = CountVectorizer(stop_words='english', max_features=2000, ngram_range = (1, ngram))
        bow = vectorizer.fit_transform(df_copy['song_lyrics'].str.lower().tolist())
        df_bow = pd.DataFrame(bow.toarray(),columns=vectorizer.get_feature_names_out())
        df_v = pd.concat([df_copy.reset_index(drop=True), df_bow.reset_index(drop=True)], axis = 1)
        df_v.drop(['song_lyrics', 'song_title','song_word_count','song_runtime', 'song_year'], axis=1, inplace=True)
        df_v.to_csv('model_input_bow_Tfid.csv', index = False)
        df_v.to_csv('model_input_bow.csv', index = False)
    elif output == 2 and tfid == True:
        vectorizer = TfidfVectorizer(stop_words='english', max_features=2000, ngram_range = (1, ngram))
        bow = vectorizer.fit_transform(df_copy['song_lyrics'].str.lower().tolist())
        df_bow = pd.DataFrame(bow.toarray(),columns=vectorizer.get_feature_names_out())
        df_v = pd.concat([df_copy.reset_index(drop=True), df_bow.reset_index(drop=True)], axis = 1)
        df_v.drop(['song_lyrics', 'song_title','song_word_count','song_runtime', 'song_year'], axis=1, inplace=True)
        df_v.to_csv('model_input_bow_Tfid.csv', index = False)
    elif output == 3 and tfid == False:
        vectorizer = CountVectorizer(stop_words='english', max_features=2000, ngram_range = (1, ngram))
        bow = vectorizer.fit_transform(df_copy['song_lyrics'].str.lower().tolist())
        df_bow = pd.DataFrame(bow.toarray(),columns=vectorizer.get_feature_names_out())
        df_bow.index = df_bow.index.astype(int) ; df_copy.index = df_copy.index.astype(int)
        df_v = pd.concat([df_copy.reset_index(drop=True), df_bow.reset_index(drop=True)], axis = 1)
        df_v.drop(['song_lyrics'], axis=1, inplace=True)
        df_v.to_csv('model_input_covariates_bow.csv', index = False)
    elif output == 3 and tfid == True:
        vectorizer = TfidfVectorizer(stop_words='english', max_features=2000, ngram_range = (1, ngram))
        bow = vectorizer.fit_transform(df_copy['song_lyrics'].str.lower().tolist())
        df_bow = pd.DataFrame(bow.toarray(),columns=vectorizer.get_feature_names_out())
        df_bow.index = df_bow.index.astype(int) ; df_copy.index = df_copy.index.astype(int)
        df_v = pd.concat([df_copy.reset_index(drop=True), df_bow.reset_index(drop=True)], axis = 1)
        df_v.drop(['song_lyrics'], axis=1, inplace=True)
        df_v.to_csv('model_input_covariates_bow_Tfid.csv', index = False)
    

In [11]:
df_l = pd.read_csv("./data/lyrics.csv")
df_g = pd.read_csv("./data/songsbyGenrePages.csv")
data_lyrics(df_l, df_g, 3, True,3)
df_l = pd.read_csv("./data/lyrics.csv")
df_g = pd.read_csv("./data/songsbyGenrePages.csv")
data_lyrics(df_l, df_g, 2, False,3)

AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names_out'

In [9]:
df_l = pd.read_csv("lyrics.csv")
df_g = pd.read_csv("songsbyGenrePages.csv")
data_lyrics(df_l, df_g, 3, False,3)
# df_l = pd.read_csv("lyrics.csv")
# df_g = pd.read_csv("songsbyGenrePages.csv")
# data_lyrics(df_l, df_g, 3, False)

C:\Users\malag\AppData\Local\Temp\ipykernel_24116\4073213511.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_l = pd.read_csv("lyrics.csv")


In [11]:
df6 = pd.read_csv("model_input_covariates.csv")
print(df6.shape)
df6.head()

(151275, 6)


,artist_name,song_title,song_runtime,song_year,song_word_count,song_genre
0,a band of bees,the rip on track,00:04:37,0,122,NaN
1,a band of bees,left foot stepdown,00:04:06,0,201,NaN
2,a band of bees,horsemen,00:03:28,0,238,NaN
3,a band of bees,chicken payback madlibs soul distortion vocal ...,00:00:00,0,283,NaN
4,a band of bees,these are the ghosts,00:03:08,0,174,NaN


In [4]:
print("Length of lyrics file: ", len(df_l))

print("Length of genre file: ", len(df_g))

print("Song Lyrics N/A (error):", df_l['song_lyrics'].isnull().sum())

sl_not_found = (df_l['song_lyrics'] == 'Not found').sum()
print("Song Lyrics not found", sl_not_found)

print("Song Lyrics not scraped:", df_l['song_lyrics'].isnull().sum() - sl_not_found)

print("Song Runtime not scraped:", (df_l['song_runtime'] == 0).sum())

print("Song Year not scraped:", df_l['song_year'].isnull().sum())

Length of lyrics file:  1123907
Length of genre file:  211032
Song Lyrics N/A (error): 389376
Song Lyrics not found 42942
Song Lyrics not scraped: 346434
Song Runtime not scraped: 75331
Song Year not scraped: 738538
